
---

# **Chapter 17: Deep Linking & Navigation 2.0**

---

## **Learning Objectives**

By the end of this chapter, you will be able to:

- Understand Navigator 2.0 and its declarative API
- Implement the Router, RouteInformationParser, and RouterDelegate
- Create declarative routing systems with URL-based navigation
- Configure deep linking on Android (App Links)
- Configure deep linking on iOS (Universal Links)
- Handle incoming deep links and route them appropriately
- Support web URLs for browser-based navigation
- Implement navigation state management with Navigator 2.0

---

## **Prerequisites**

- Completed Chapter 16: Navigation Fundamentals
- Understanding of Navigator 1.0 (push, pop, named routes)
- Knowledge of BuildContext and widget lifecycle
- Basic familiarity with AndroidManifest.xml and iOS configuration
- Understanding of URL structures and web concepts

---

## **17.1 Understanding Navigator 2.0**

Navigator 2.0 is Flutter's declarative navigation API that provides full control over the navigation stack and supports deep linking, web URLs, and complex navigation patterns.

### **Navigator 1.0 vs Navigator 2.0**

```dart
import 'package:flutter/material.dart';

// ============================================
// NAVIGATOR 1.0 (IMPERATIVE)
// ============================================
// Navigator 1.0 uses imperative commands:
// - push(), pop(), pushNamed(), etc.
// - The navigation state is managed internally
// - Hard to sync with the URL (on web)
// - Difficult to handle deep links

class Navigator1Example extends StatelessWidget {
  const Navigator1Example({super.key});

  @override
  Widget build(BuildContext context) {
    return MaterialApp(
      title: 'Navigator 1.0',
      initialRoute: '/',
      // Routes map for named navigation
      routes: {
        '/': (context) => const HomeScreen(),
        '/details': (context) => const DetailsScreen(),
        '/profile': (context) => const ProfileScreen(),
      },
    );
  }
}

class HomeScreen extends StatelessWidget {
  const HomeScreen({super.key});

  @override
  Widget build(BuildContext context) {
    return Scaffold(
      appBar: AppBar(title: const Text('Home')),
      body: Center(
        child: ElevatedButton(
          // Imperative navigation: push a named route
          onPressed: () => Navigator.pushNamed(context, '/details'),
          child: const Text('Go to Details'),
        ),
      ),
    );
  }
}

class DetailsScreen extends StatelessWidget {
  const DetailsScreen({super.key});

  @override
  Widget build(BuildContext context) {
    return Scaffold(
      appBar: AppBar(title: const Text('Details')),
      body: Center(
        child: ElevatedButton(
          // Imperative navigation: pop the current route
          onPressed: () => Navigator.pop(context),
          child: const Text('Go Back'),
        ),
      ),
    );
  }
}

class ProfileScreen extends StatelessWidget {
  const ProfileScreen({super.key});

  @override
  Widget build(BuildContext context) {
    return Scaffold(
      appBar: AppBar(title: const Text('Profile')),
      body: const Center(
        child: Text('Profile Screen'),
      ),
    );
  }
}

// ============================================
// NAVIGATOR 2.0 (DECLARATIVE)
// ============================================
// Navigator 2.0 uses a declarative approach:
// - You define the navigation state as data
// - Flutter updates the UI when the state changes
// - Navigation state can be synced with the URL
// - Full control over the navigation stack
// - Perfect for deep linking and web support

class Navigator2Example extends StatelessWidget {
  const Navigator2Example({super.key});

  @override
  Widget build(BuildContext context) {
    return MaterialApp.router(
      title: 'Navigator 2.0',
      // Instead of 'home' or 'routes', we use:
      // - routerDelegate: Builds the navigation stack
      // - routeInformationParser: Parses the URL/URI
      // - backButtonDispatcher: Handles back button (Android)
      
      routerDelegate: MyAppRouterDelegate(),
      // The routerDelegate is responsible for building the Navigator
      // It listens to changes in the navigation state and rebuilds accordingly
      
      routeInformationParser: MyAppRouteInformationParser(),
      // The routeInformationParser converts RouteInformation (URL)
      // into your custom navigation state (e.g., a list of pages)
      
      backButtonDispatcher: RootBackButtonDispatcher(),
      // Handles the Android back button
      // Can be customized to modify back button behavior
    );
  }
}
```

**Explanation:**

- **Navigator 1.0 (Imperative)**: The traditional navigation approach in Flutter uses imperative commands like `push()`, `pop()`, and `pushNamed()`. You tell Flutter "push this route" or "pop that route." The navigation state is managed internally by the Navigator, and you don't have direct access to it. This approach works well for simple apps but has limitations:
  - Hard to sync with the browser URL on web
  - Difficult to handle deep links from external sources
  - No easy way to inspect or manipulate the entire navigation stack
  - Cannot express the navigation state as data

- **Navigator 2.0 (Declarative)**: The new navigation API uses a declarative approach. Instead of issuing commands, you define your navigation state as data (e.g., a list of pages). When the state changes, Flutter automatically updates the UI. This provides several benefits:
  - **Full control**: You have complete access to the navigation stack
  - **URL sync**: Easy to keep the browser URL in sync with navigation state (web support)
  - **Deep linking**: Simple to handle incoming deep links and route them appropriately
  - **Predictable**: Navigation is driven by state changes, making it easier to understand and debug
  - **Testable**: Since navigation is driven by data, it's easier to write tests

- **`MaterialApp.router()`**: Instead of `MaterialApp` with `home` or `routes`, use `MaterialApp.router()`. This constructor takes three key parameters for Navigator 2.0:
  - `routerDelegate`: Builds and manages the Navigator based on the navigation state
  - `routeInformationParser`: Converts URL/RouteInformation into your custom navigation state
  - `backButtonDispatcher`: Handles the Android back button (and can be customized)

- **`routerDelegate`**: An instance of a class that implements `RouterDelegate<T>`. It's responsible for:
  - Building the Navigator with the current pages
  - Listening to navigation state changes and rebuilding
  - Handling navigation events (like pushing/popping pages)
  - Updating the URL when navigation state changes (on web)

- **`routeInformationParser`**: An instance of a class that implements `RouteInformationParser<T>`. It's responsible for:
  - Parsing the current URL (from the browser or deep link)
  - Converting it into your custom navigation state (type `T`)
  - Handling URL updates when navigation state changes

- **`backButtonDispatcher`**: Handles the hardware back button on Android. The default `RootBackButtonDispatcher()` routes back button presses to the active Navigator. You can customize this to modify back button behavior.

### **Core Components of Navigator 2.0**

Navigator 2.0 consists of three main components that work together to manage navigation declaratively.

```dart
import 'package:flutter/material.dart';

// ============================================
// NAVIGATION STATE (THE DATA MODEL)
// ============================================
// First, define your navigation state as a data structure
// This represents the entire navigation stack

enum AppTab { home, profile, settings }

class AppState {
  // List of pages currently in the navigation stack
  // Each page is represented by a PageConfig
  final List<PageConfig> pages;
  
  // Currently selected tab (for bottom navigation)
  final AppTab selectedTab;
  
  const AppState({
    this.pages = const [PageConfig.home()],
    this.selectedTab = AppTab.home,
  });
  
  // Factory constructor for creating the initial state
  factory AppState.initial() {
    return const AppState(
      pages: [PageConfig.home()],
      selectedTab: AppTab.home,
    );
  }
  
  // CopyWith method for creating new states
  // This is a common pattern in Flutter for immutable state
  AppState copyWith({
    List<PageConfig>? pages,
    AppTab? selectedTab,
  }) {
    return AppState(
      pages: pages ?? this.pages,
      selectedTab: selectedTab ?? this.selectedTab,
    );
  }
}

// PageConfig represents a single page in the navigation stack
// It contains all the information needed to build a Page widget
class PageConfig {
  final String key; // Unique key for the page
  final String name; // Route name (e.g., '/', '/details')
  final Object? arguments; // Optional arguments passed to the page
  
  const PageConfig({
    required this.key,
    required this.name,
    this.arguments,
  });
  
  // Factory constructors for common pages
  factory PageConfig.home() {
    return const PageConfig(
      key: 'Home',
      name: '/',
    );
  }
  
  factory PageConfig.details(String itemId) {
    return PageConfig(
      key: 'Details-$itemId',
      name: '/details',
      arguments: itemId,
    );
  }
  
  factory PageConfig.profile() {
    return const PageConfig(
      key: 'Profile',
      name: '/profile',
    );
  }
  
  factory PageConfig.settings() {
    return const PageConfig(
      key: 'Settings',
      name: '/settings',
    );
  }
  
  // Convert PageConfig to a Page widget
  // This is called by the RouterDelegate to build the actual pages
  Page toPage() {
    switch (name) {
      case '/':
        return MaterialPage(
          key: ValueKey(key),
          child: const HomeScreen(),
        );
      case '/details':
        return MaterialPage(
          key: ValueKey(key),
          child: DetailsScreen(arguments: arguments),
        );
      case '/profile':
        return MaterialPage(
          key: ValueKey(key),
          child: const ProfileScreen(),
        );
      case '/settings':
        return MaterialPage(
          key: ValueKey(key),
          child: const SettingsScreen(),
        );
      default:
        return MaterialPage(
          key: ValueKey(key),
          child: NotFoundScreen(name: name),
        );
    }
  }
  
  // Convert PageConfig to a URL path
  // This is used for syncing with the browser URL (on web)
  String toPath() {
    switch (name) {
      case '/':
        return '/';
      case '/details':
        final itemId = arguments as String?;
        return itemId != null ? '/details/$itemId' : '/details';
      case '/profile':
        return '/profile';
      case '/settings':
        return '/settings';
      default:
        return name;
    }
  }
}

// ============================================
// ROUTE INFORMATION PARSER
// ============================================
// Converts RouteInformation (URL) into AppState (navigation state)
// Also converts AppState back to RouteInformation (for URL updates)

class MyAppRouteInformationParser extends RouteInformationParser<AppState> {
  @override
  // Parse the incoming RouteInformation into AppState
  // Called when:
  // - The app starts (from initial URL)
  // - The user changes the URL in the browser (web)
  // - A deep link is received (from external app)
  Future<AppState> parseRouteInformation(RouteInformation information) async {
    final uri = information.uri;
    print('Parsing URL: $uri');
    
    // Parse the URI path and create a corresponding AppState
    final path = uri.path;
    
    switch (path) {
      case '/':
        return AppState.initial();
        
      case '/profile':
        return const AppState(
          pages: [PageConfig.profile()],
          selectedTab: AppTab.profile,
        );
        
      case '/settings':
        return const AppState(
          pages: [PageConfig.settings()],
          selectedTab: AppTab.settings,
        );
        
      default:
        // Check if it's a details page with an ID
        // Pattern: /details/:id
        if (path.startsWith('/details/')) {
          final parts = path.split('/');
          if (parts.length >= 3) {
            final itemId = parts[2];
            return AppState(
              pages: [PageConfig.details(itemId)],
              selectedTab: AppTab.home,
            );
          }
        }
        
        // Unknown route - show 404
        return AppState(
          pages: [PageConfig(key: 'NotFound-$path', name: path)],
          selectedTab: AppTab.home,
        );
    }
  }
  
  @override
  // Restore state from saved data (e.g., after app restart)
  // Called when the app is restored from a saved state
  Future<AppState> restoreRouteInformation(AppState configuration) async {
    // Simply return the saved state
    return configuration;
  }
  
  @override
  // Convert AppState back to RouteInformation
  // Called when the navigation state changes (to update the URL)
  RouteInformation? restoreRouteInformation(AppState configuration) {
    // Get the last page's path
    final lastPage = configuration.pages.last;
    final path = lastPage.toPath();
    
    return RouteInformation(
      uri: Uri.parse(path),
      // You can also include state in the URI (for hash-based routing)
      // state: configuration.toString(),
    );
  }
}

// ============================================
// ROUTER DELEGATE
// ============================================
// Builds the Navigator based on the current AppState
// Handles navigation events and updates the state

class MyAppRouterDelegate extends RouterDelegate<AppState>
    with ChangeNotifier, PopNavigatorRouterDelegateMixin<AppState> {
  // Current navigation state
  AppState _currentState = AppState.initial();
  
  // Getter for the current state
  AppState get currentState => _currentState;
  
  // Getter for the navigator key (required by PopNavigatorRouterDelegateMixin)
  @override
  final GlobalKey<NavigatorState> navigatorKey;
  
  // Constructor
  MyAppRouterDelegate() : navigatorKey = GlobalKey<NavigatorState>();
  
  // Update the current state and notify listeners
  // This triggers a rebuild of the Navigator
  void updateState(AppState newState) {
    _currentState = newState;
    notifyListeners();
    // notifyListeners() is required by ChangeNotifier
    // It tells all listeners (including the Router) that the state has changed
  }
  
  // Push a new page onto the stack
  void pushPage(PageConfig page) {
    final newPages = [..._currentState.pages, page];
    updateState(_currentState.copyWith(pages: newPages));
  }
  
  // Pop the current page from the stack
  void popPage() {
    if (_currentState.pages.length > 1) {
      final newPages = _currentState.pages.sublist(0, _currentState.pages.length - 1);
      updateState(_currentState.copyWith(pages: newPages));
    }
  }
  
  // Replace the entire stack with a single page
  void replaceAll(PageConfig page) {
    updateState(_currentState.copyWith(pages: [page]));
  }
  
  // ============================================
  // ROUTER DELEGATE OVERRIDES
  // ============================================
  
  @override
  // Build the widget tree for the current state
  // Called whenever the state changes
  Widget build(BuildContext context) {
    print('Building navigator with ${_currentState.pages.length} pages');
    
    return Navigator(
      key: navigatorKey,
      // pages: List of Page widgets representing the navigation stack
      // The Navigator displays these pages in order (from bottom to top)
      pages: _currentState.pages.map((config) => config.toPage()).toList(),
      
      // onPopPage: Called when the user tries to go back
      // Return true if the page was successfully popped
      // Return false to prevent the pop
      onPopPage: (route, result) {
        // If there's more than one page, allow the pop
        if (_currentState.pages.length > 1) {
          popPage();
          return true;
        }
        // If there's only one page, don't allow the pop
        // (This prevents closing the app from the home screen)
        return false;
      },
    );
  }
  
  @override
  // Get the current configuration (navigation state)
  // Called by the Router to get the current state
  AppState? get currentConfiguration => _currentState;
  
  @override
  // Handle initial route parsing
  // Called when the app starts to parse the initial URL
  Future<void> setInitialRoutePath(AppState configuration) async {
    // The initial state is already set in the constructor
    // This method is called after parseRouteInformation
    // We don't need to do anything here
  }
  
  @override
  // Handle new route information from the parser
  // Called when the URL changes (from browser or deep link)
  Future<void> setNewRoutePath(AppState configuration) async {
    // Update the state with the new configuration
    _currentState = configuration;
    notifyListeners();
  }
}

// ============================================
// SCREEN WIDGETS
// ============================================

class HomeScreen extends StatelessWidget {
  const HomeScreen({super.key});

  @override
  Widget build(BuildContext context) {
    // Get the RouterDelegate to access navigation methods
    final routerDelegate = Router.of(context).routerDelegate as MyAppRouterDelegate;
    
    return Scaffold(
      appBar: AppBar(title: const Text('Home')),
      body: Center(
        child: Column(
          mainAxisAlignment: MainAxisAlignment.center,
          children: [
            ElevatedButton(
              onPressed: () {
                // Push the details page with an ID
                routerDelegate.pushPage(PageConfig.details('123'));
              },
              child: const Text('Go to Details'),
            ),
            const SizedBox(height: 16),
            ElevatedButton(
              onPressed: () {
                // Navigate to profile (replace stack)
                routerDelegate.replaceAll(PageConfig.profile());
              },
              child: const Text('Go to Profile'),
            ),
          ],
        ),
      ),
    );
  }
}

class DetailsScreen extends StatelessWidget {
  final Object? arguments;
  
  const DetailsScreen({super.key, this.arguments});

  @override
  Widget build(BuildContext context) {
    final routerDelegate = Router.of(context).routerDelegate as MyAppRouterDelegate;
    
    return Scaffold(
      appBar: AppBar(title: const Text('Details')),
      body: Center(
        child: Column(
          mainAxisAlignment: MainAxisAlignment.center,
          children: [
            Text('Item ID: ${arguments ?? "None"}'),
            const SizedBox(height: 16),
            ElevatedButton(
              onPressed: () {
                // Push another details page
                routerDelegate.pushPage(PageConfig.details('456'));
              },
              child: const Text('Go to Another Details'),
            ),
            const SizedBox(height: 16),
            ElevatedButton(
              onPressed: () {
                // Pop back to previous screen
                routerDelegate.popPage();
              },
              child: const Text('Go Back'),
            ),
          ],
        ),
      ),
    );
  }
}

class ProfileScreen extends StatelessWidget {
  const ProfileScreen({super.key});

  @override
  Widget build(BuildContext context) {
    return Scaffold(
      appBar: AppBar(title: const Text('Profile')),
      body: const Center(
        child: Text('Profile Screen'),
      ),
    );
  }
}

class SettingsScreen extends StatelessWidget {
  const SettingsScreen({super.key});

  @override
  Widget build(BuildContext context) {
    return Scaffold(
      appBar: AppBar(title: const Text('Settings')),
      body: const Center(
        child: Text('Settings Screen'),
      ),
    );
  }
}

class NotFoundScreen extends StatelessWidget {
  final String name;
  
  const NotFoundScreen({super.key, required this.name});

  @override
  Widget build(BuildContext context) {
    return Scaffold(
      appBar: AppBar(title: const Text('Not Found')),
      body: Center(
        child: Text('404: $name'),
      ),
    );
  }
}
```

**Explanation:**

- **`AppState`**: The central data model that represents the entire navigation state. It contains:
  - `pages`: A list of `PageConfig` objects, each representing a page in the navigation stack. The first page is the bottom of the stack (the home screen), and the last page is the top of the stack (the currently visible screen).
  - `selectedTab`: The currently selected tab (for bottom navigation). This demonstrates that your navigation state can include more than just the page stack.
  - `copyWith()`: A method for creating new `AppState` objects with some fields changed. Since the state is immutable, this is the only way to update it. This is a common pattern in Flutter for state management.

- **`PageConfig`**: Represents a single page in the navigation stack. It contains:
  - `key`: A unique identifier for the page. This is used by Flutter to identify pages and determine whether a page needs to be rebuilt or reused. Keys should be unique across all pages.
  - `name`: The route name (e.g., `'/'`, `'/details'`, `'/profile'`). This corresponds to a URL path.
  - `arguments`: Optional data passed to the page (e.g., an item ID for a details page).
  - Factory constructors: Convenience methods for creating common pages (e.g., `PageConfig.home()`, `PageConfig.details('123')`).
  - `toPage()`: Converts the `PageConfig` into a `Page` widget (typically `MaterialPage` or `CupertinoPage`). This is called by the `RouterDelegate` when building the Navigator.
  - `toPath()`: Converts the `PageConfig` into a URL path. This is used for syncing with the browser URL on web.

- **`RouteInformationParser`**: Converts between `RouteInformation` (which contains a URL and optional state) and your custom navigation state (`AppState`). The `RouteInformationParser<AppState>` class requires implementing:
  - `parseRouteInformation()`: Parses incoming URL/URI into an `AppState`. Called when the app starts, when the browser URL changes, or when a deep link is received.
  - `restoreRouteInformation()`: Restores state from saved data (e.g., after app restart). This is used for state preservation across app restarts.
  - `restoreRouteInformation()` (with `RouteInformation?` return): Converts `AppState` back to `RouteInformation`. Called when the navigation state changes, to update the browser URL.

- **`RouterDelegate`**: Builds and manages the Navigator based on the current navigation state. The `RouterDelegate<AppState>` mixin (combined with `ChangeNotifier` and `PopNavigatorRouterDelegateMixin`) requires implementing:
  - `build()`: Builds the Navigator widget with the current pages. Called whenever the state changes (after `notifyListeners()` is called).
  - `currentConfiguration`: Returns the current navigation state (`AppState`). Called by the Router to get the current state for URL updates.
  - `setInitialRoutePath()`: Handles the initial route parsing. Called when the app starts, after `parseRouteInformation()`.
  - `setNewRoutePath()`: Handles new route information from the parser. Called when the URL changes (from browser or deep link).
  - `navigatorKey`: A `GlobalKey<NavigatorState>` that identifies the Navigator. Required by `PopNavigatorRouterDelegateMixin` for handling back button presses.

- **`ChangeNotifier` mixin**: Provides `notifyListeners()` method, which notifies all listeners when the state changes. The `Router` listens to the `RouterDelegate` and rebuilds when notified.

- **`PopNavigatorRouterDelegateMixin<AppState>` mixin**: Provides functionality for handling back button presses. It requires a `navigatorKey` property.

- **Navigation methods**: The `RouterDelegate` provides methods for manipulating the navigation stack:
  - `pushPage()`: Adds a new page to the stack. Creates a new state with the page appended and calls `notifyListeners()`.
  - `popPage()`: Removes the top page from the stack. Creates a new state with the last page removed and calls `notifyListeners()`.
  - `replaceAll()`: Replaces the entire stack with a single page. Creates a new state with only the specified page and calls `notifyListeners()`.

- **`Navigator` widget**: Built in the `build()` method, it displays the list of pages. The `onPopPage` callback is called when the user tries to go back (via back button or swipe). If it returns `true`, the page is popped. If it returns `false`, the pop is prevented.

- **Accessing the RouterDelegate**: In screen widgets, you can access the `RouterDelegate` using `Router.of(context).routerDelegate` and cast it to your custom type. This allows screens to trigger navigation programmatically.

---

## **17.2 Declarative Routing with URL Support**

Navigator 2.0 enables full URL support, allowing your Flutter web app to have meaningful URLs that can be bookmarked, shared, and deep-linked.

### **Complete Declarative Routing Implementation**

```dart
import 'package:flutter/material.dart';

// ============================================
// MAIN APP WITH DECLARATIVE ROUTING
// ============================================

void main() {
  runApp(const DeclarativeRoutingApp());
}

class DeclarativeRoutingApp extends StatelessWidget {
  const DeclarativeRoutingApp({super.key});

  @override
  Widget build(BuildContext context) {
    return MaterialApp.router(
      title: 'Declarative Routing',
      debugShowCheckedModeBanner: false,
      
      // Theme configuration
      theme: ThemeData(
        useMaterial3: true,
        colorSchemeSeed: Colors.blue,
      ),
      
      // Router delegate - manages the navigation stack
      routerDelegate: AppRouterDelegate(),
      
      // Route information parser - parses URLs into state
      routeInformationParser: AppRouteInformationParser(),
      
      // Back button dispatcher - handles Android back button
      backButtonDispatcher: RootBackButtonDispatcher(),
    );
  }
}

// ============================================
// NAVIGATION STATE
// ============================================

// Enum for app tabs
enum AppTab { home, search, profile }

// Navigation state class
class AppNavigationState {
  // List of pages in the navigation stack
  final List<AppPage> pages;
  
  // Currently selected tab
  final AppTab selectedTab;
  
  // Whether the app is in a "tab switching" state
  // (used to determine whether to animate transitions)
  final bool isTabSwitch;
  
  const AppNavigationState({
    this.pages = const [AppPage.home()],
    this.selectedTab = AppTab.home,
    this.isTabSwitch = false,
  });
  
  // Initial state
  factory AppNavigationState.initial() {
    return const AppNavigationState(
      pages: [AppPage.home()],
      selectedTab: AppTab.home,
    );
  }
  
  // CopyWith for immutable updates
  AppNavigationState copyWith({
    List<AppPage>? pages,
    AppTab? selectedTab,
    bool? isTabSwitch,
  }) {
    return AppNavigationState(
      pages: pages ?? this.pages,
      selectedTab: selectedTab ?? this.selectedTab,
      isTabSwitch: isTabSwitch ?? this.isTabSwitch,
    );
  }
  
  // Get the current page (top of the stack)
  AppPage get currentPage => pages.last;
  
  // Get the home page (bottom of the stack)
  AppPage get homePage => pages.first;
  
  // Check if we're on the home page
  bool get isHomePage => pages.length == 1 && pages.first.isHome;
}

// Page configuration class
class AppPage {
  final String key;
  final String path;
  final String title;
  final Object? arguments;
  
  const AppPage({
    required this.key,
    required this.path,
    required this.title,
    this.arguments,
  });
  
  // Factory constructors for common pages
  factory AppPage.home() {
    return const AppPage(
      key: 'home',
      path: '/',
      title: 'Home',
    );
  }
  
  factory AppPage.search() {
    return const AppPage(
      key: 'search',
      path: '/search',
      title: 'Search',
    );
  }
  
  factory AppPage.profile() {
    return const AppPage(
      key: 'profile',
      path: '/profile',
      title: 'Profile',
    );
  }
  
  factory AppPage.product(String productId) {
    return AppPage(
      key: 'product-$productId',
      path: '/product/$productId',
      title: 'Product',
      arguments: productId,
    );
  }
  
  factory AppPage.category(String categoryId) {
    return AppPage(
      key: 'category-$categoryId',
      path: '/category/$categoryId',
      title: 'Category',
      arguments: categoryId,
    );
  }
  
  factory AppPage.settings() {
    return const AppPage(
      key: 'settings',
      path: '/settings',
      title: 'Settings',
    );
  }
  
  // Check if this is the home page
  bool get isHome => path == '/';
  
  // Check if this is a tab page
  bool get isTabPage => path == '/' || path == '/search' || path == '/profile';
  
  // Convert to a Page widget
  Page<dynamic> toPage() {
    // Use MaterialPage for Material Design
    // You could use CupertinoPage for iOS-style pages
    return MaterialPage(
      key: ValueKey(key),
      child: _buildPage(),
      // You can also specify:
      // - name: The route name (for debugging)
      // - arguments: The arguments to pass to the page
      // - restorationId: For state restoration
    );
  }
  
  // Build the page widget
  Widget _buildPage() {
    switch (path) {
      case '/':
        return const HomeScreen();
      case '/search':
        return const SearchScreen();
      case '/profile':
        return const ProfileScreen();
      case '/settings':
        return const SettingsScreen();
      default:
        // Check if it's a product page
        if (path.startsWith('/product/')) {
          final productId = arguments as String?;
          return ProductScreen(productId: productId ?? 'unknown');
        }
        
        // Check if it's a category page
        if (path.startsWith('/category/')) {
          final categoryId = arguments as String?;
          return CategoryScreen(categoryId: categoryId ?? 'unknown');
        }
        
        // 404 page
        return NotFoundScreen(path: path);
    }
  }
  
  // Convert to a URL path
  String toPath() {
    return path;
  }
}

// ============================================
// ROUTE INFORMATION PARSER
// ============================================

class AppRouteInformationParser extends RouteInformationParser<AppNavigationState> {
  @override
  Future<AppNavigationState> parseRouteInformation(
    RouteInformation information,
  ) async {
    final uri = information.uri;
    print('Parsing route: $uri');
    
    final path = uri.path;
    final queryParameters = uri.queryParameters;
    
    // Determine the selected tab based on the path
    AppTab selectedTab = AppTab.home;
    if (path == '/search') {
      selectedTab = AppTab.search;
    } else if (path == '/profile') {
      selectedTab = AppTab.profile;
    }
    
    // Parse the path and create the navigation state
    List<AppPage> pages = [];
    
    switch (path) {
      case '/':
        pages = [AppPage.home()];
        break;
        
      case '/search':
        pages = [AppPage.search()];
        break;
        
      case '/profile':
        pages = [AppPage.profile()];
        break;
        
      case '/settings':
        pages = [AppPage.home(), AppPage.settings()];
        break;
        
      default:
        // Parse product page: /product/:id
        if (path.startsWith('/product/')) {
          final parts = path.split('/');
          if (parts.length >= 3) {
            final productId = parts[2];
            pages = [AppPage.home(), AppPage.product(productId)];
            selectedTab = AppTab.home;
          }
        }
        // Parse category page: /category/:id
        else if (path.startsWith('/category/')) {
          final parts = path.split('/');
          if (parts.length >= 3) {
            final categoryId = parts[2];
            pages = [AppPage.home(), AppPage.category(categoryId)];
            selectedTab = AppTab.home;
          }
        }
        // 404
        else {
          pages = [AppPage(key: '404-$path', path: path, title: 'Not Found')];
        }
    }
    
    return AppNavigationState(
      pages: pages,
      selectedTab: selectedTab,
    );
  }
  
  @override
  Future<AppNavigationState> restoreRouteInformation(
    AppNavigationState configuration,
  ) async {
    // Simply return the saved configuration
    // This is called when restoring the app from a saved state
    return configuration;
  }
  
  @override
  RouteInformation? restoreRouteInformation(
    AppNavigationState configuration,
  ) {
    // Convert the current navigation state back to a RouteInformation
    // This updates the browser URL when the navigation state changes
    final path = configuration.currentPage.toPath();
    
    return RouteInformation(
      uri: Uri.parse(path),
      // You can also include state in the RouteInformation
      // state: configuration.toString(),
    );
  }
}

// ============================================
// ROUTER DELEGATE
// ============================================

class AppRouterDelegate extends RouterDelegate<AppNavigationState>
    with ChangeNotifier, PopNavigatorRouterDelegateMixin<AppNavigationState> {
  // Current navigation state
  AppNavigationState _state = AppNavigationState.initial();
  
  // Getter for the current state
  AppNavigationState get state => _state;
  
  // Getter for the current page
  AppPage get currentPage => _state.currentPage;
  
  // Navigator key (required by PopNavigatorRouterDelegateMixin)
  @override
  final GlobalKey<NavigatorState> navigatorKey = GlobalKey<NavigatorState>();
  
  // Update the state and notify listeners
  void _updateState(AppNavigationState newState) {
    _state = newState;
    notifyListeners();
    print('State updated: ${_state.pages.map((p) => p.path).join(' -> ')}');
  }
  
  // Push a new page
  void pushPage(AppPage page) {
    final newPages = [..._state.pages, page];
    _updateState(_state.copyWith(pages: newPages, isTabSwitch: false));
  }
  
  // Pop the current page
  bool popPage() {
    if (_state.pages.length > 1) {
      final newPages = _state.pages.sublist(0, _state.pages.length - 1);
      
      // Update selected tab if needed
      AppTab newTab = _state.selectedTab;
      if (newPages.last.isTabPage) {
        switch (newPages.last.path) {
          case '/':
            newTab = AppTab.home;
            break;
          case '/search':
            newTab = AppTab.search;
            break;
          case '/profile':
            newTab = AppTab.profile;
            break;
        }
      }
      
      _updateState(_state.copyWith(
        pages: newPages,
        selectedTab: newTab,
        isTabSwitch: false,
      ));
      return true;
    }
    return false;
  }
  
  // Replace all pages with a single page
  void replaceAll(AppPage page) {
    AppTab newTab = AppTab.home;
    if (page.isTabPage) {
      switch (page.path) {
        case '/':
          newTab = AppTab.home;
          break;
        case '/search':
          newTab = AppTab.search;
          break;
        case '/profile':
          newTab = AppTab.profile;
          break;
      }
    }
    
    _updateState(_state.copyWith(
      pages: [page],
      selectedTab: newTab,
      isTabSwitch: false,
    ));
  }
  
  // Switch to a tab (keep tab's page history)
  void switchTab(AppTab tab) {
    AppPage tabPage;
    switch (tab) {
      case AppTab.home:
        tabPage = AppPage.home();
        break;
      case AppTab.search:
        tabPage = AppPage.search();
        break;
      case AppTab.profile:
        tabPage = AppPage.profile();
        break;
    }
    
    _updateState(_state.copyWith(
      pages: [tabPage],
      selectedTab: tab,
      isTabSwitch: true,
    ));
  }
  
  // Push a tab page (if not already on a tab page)
  void pushTabPage(AppPage page) {
    // Check if we're already on a tab page
    if (_state.currentPage.isTabPage) {
      // Just replace with the new tab page
      AppTab newTab = AppTab.home;
      switch (page.path) {
        case '/':
          newTab = AppTab.home;
          break;
        case '/search':
          newTab = AppTab.search;
          break;
        case '/profile':
          newTab = AppTab.profile;
          break;
      }
      
      _updateState(_state.copyWith(
        pages: [page],
        selectedTab: newTab,
        isTabSwitch: true,
      ));
    } else {
      // Push the tab page on top
      AppTab newTab = AppTab.home;
      switch (page.path) {
        case '/':
          newTab = AppTab.home;
          break;
        case '/search':
          newTab = AppTab.search;
          break;
        case '/profile':
          newTab = AppTab.profile;
          break;
      }
      
      final newPages = [..._state.pages, page];
      _updateState(_state.copyWith(
        pages: newPages,
        selectedTab: newTab,
        isTabSwitch: true,
      ));
    }
  }
  
  // ============================================
  // ROUTER DELEGATE OVERRIDES
  // ============================================
  
  @override
  Widget build(BuildContext context) {
    print('Building navigator with ${_state.pages.length} pages');
    
    return Navigator(
      key: navigatorKey,
      pages: _state.pages.map((page) => page.toPage()).toList(),
      onPopPage: (route, result) {
        // Allow pop if there's more than one page
        if (_state.pages.length > 1) {
          popPage();
          route.didPop(result);
          return true;
        }
        return false;
      },
    );
  }
  
  @override
  AppNavigationState? get currentConfiguration => _state;
  
  @override
  Future<void> setInitialRoutePath(AppNavigationState configuration) async {
    // The initial state is already set
    // No action needed
  }
  
  @override
  Future<void> setNewRoutePath(AppNavigationState configuration) async {
    // Update the state with the new configuration from the parser
    _state = configuration;
    notifyListeners();
  }
}

// ============================================
// SCREEN WIDGETS
// ============================================

class HomeScreen extends StatelessWidget {
  const HomeScreen({super.key});

  @override
  Widget build(BuildContext context) {
    return Scaffold(
      appBar: AppBar(title: const Text('Home')),
      body: ListView(
        children: [
          ListTile(
            title: const Text('Product: Laptop'),
            subtitle: const Text('View product details'),
            onTap: () {
              final delegate = Router.of(context).routerDelegate as AppRouterDelegate;
              delegate.pushPage(AppPage.product('laptop-123'));
            },
          ),
          ListTile(
            title: const Text('Product: Smartphone'),
            subtitle: const Text('View product details'),
            onTap: () {
              final delegate = Router.of(context).routerDelegate as AppRouterDelegate;
              delegate.pushPage(AppPage.product('phone-456'));
            },
          ),
          ListTile(
            title: const Text('Category: Electronics'),
            subtitle: const Text('Browse electronics'),
            onTap: () {
              final delegate = Router.of(context).routerDelegate as AppRouterDelegate;
              delegate.pushPage(AppPage.category('electronics'));
            },
          ),
          ListTile(
            title: const Text('Category: Clothing'),
            subtitle: const Text('Browse clothing'),
            onTap: () {
              final delegate = Router.of(context).routerDelegate as AppRouterDelegate;
              delegate.pushPage(AppPage.category('clothing'));
            },
          ),
        ],
      ),
    );
  }
}

class SearchScreen extends StatelessWidget {
  const SearchScreen({super.key});

  @override
  Widget build(BuildContext context) {
    return Scaffold(
      appBar: AppBar(
        title: const Text('Search'),
        actions: [
          IconButton(
            icon: const Icon(Icons.settings),
            onPressed: () {
              final delegate = Router.of(context).routerDelegate as AppRouterDelegate;
              delegate.pushPage(AppPage.settings());
            },
          ),
        ],
      ),
      body: const Center(
        child: Text('Search Screen'),
      ),
    );
  }
}

class ProfileScreen extends StatelessWidget {
  const ProfileScreen({super.key});

  @override
  Widget build(BuildContext context) {
    return Scaffold(
      appBar: AppBar(
        title: const Text('Profile'),
        actions: [
          IconButton(
            icon: const Icon(Icons.settings),
            onPressed: () {
              final delegate = Router.of(context).routerDelegate as AppRouterDelegate;
              delegate.pushPage(AppPage.settings());
            },
          ),
        ],
      ),
      body: const Center(
        child: Text('Profile Screen'),
      ),
    );
  }
}

class ProductScreen extends StatelessWidget {
  final String? productId;
  
  const ProductScreen({super.key, this.productId});

  @override
  Widget build(BuildContext context) {
    final delegate = Router.of(context).routerDelegate as AppRouterDelegate;
    
    return Scaffold(
      appBar: AppBar(title: Text('Product: $productId')),
      body: Center(
        child: Column(
          mainAxisAlignment: MainAxisAlignment.center,
          children: [
            Text('Product ID: $productId'),
            const SizedBox(height: 20),
            ElevatedButton(
              onPressed: () => delegate.pushPage(AppPage.product('product-789')),
              child: const Text('View Another Product'),
            ),
            const SizedBox(height: 20),
            ElevatedButton(
              onPressed: () => delegate.popPage(),
              child: const Text('Go Back'),
            ),
          ],
        ),
      ),
    );
  }
}

class CategoryScreen extends StatelessWidget {
  final String? categoryId;
  
  const CategoryScreen({super.key, this.categoryId});

  @override
  Widget build(BuildContext context) {
    final delegate = Router.of(context).routerDelegate as AppRouterDelegate;
    
    return Scaffold(
      appBar: AppBar(title: Text('Category: $categoryId')),
      body: Center(
        child: Column(
          mainAxisAlignment: MainAxisAlignment.center,
          children: [
            Text('Category ID: $categoryId'),
            const SizedBox(height: 20),
            ElevatedButton(
              onPressed: () => delegate.popPage(),
              child: const Text('Go Back'),
            ),
          ],
        ),
      ),
    );
  }
}

class SettingsScreen extends StatelessWidget {
  const SettingsScreen({super.key});

  @override
  Widget build(BuildContext context) {
    final delegate = Router.of(context).routerDelegate as AppRouterDelegate;
    
    return Scaffold(
      appBar: AppBar(title: const Text('Settings')),
      body: Center(
        child: ElevatedButton(
          onPressed: () => delegate.popPage(),
          child: const Text('Go Back'),
        ),
      ),
    );
  }
}

class NotFoundScreen extends StatelessWidget {
  final String path;
  
  const NotFoundScreen({super.key, required this.path});

  @override
  Widget build(BuildContext context) {
    final delegate = Router.of(context).routerDelegate as AppRouterDelegate;
    
    return Scaffold(
      appBar: AppBar(title: const Text('Not Found')),
      body: Center(
        child: Column(
          mainAxisAlignment: MainAxisAlignment.center,
          children: [
            const Icon(Icons.error_outline, size: 64, color: Colors.red),
            const SizedBox(height: 16),
            Text('404: $path'),
            const SizedBox(height: 20),
            ElevatedButton(
              onPressed: () => delegate.replaceAll(AppPage.home()),
              child: const Text('Go Home'),
            ),
          ],
        ),
      ),
    );
  }
}
```

**Explanation:**

- **Complete declarative routing**: This implementation provides a full-featured declarative routing system with URL support. It includes:
  - Navigation state management (`AppNavigationState`)
  - Page configuration (`AppPage`)
  - Route parsing (`AppRouteInformationParser`)
  - Router delegate (`AppRouterDelegate`)
  - Screen widgets

- **`AppNavigationState` enhancements**: In addition to the page stack, this state includes:
  - `selectedTab`: Tracks the currently selected tab for bottom navigation
  - `isTabSwitch`: A flag that indicates whether the current navigation event is a tab switch. This can be used to customize transitions (e.g., no animation for tab switches).

- **`AppPage` enhancements**: Each page includes:
  - `key`: Unique identifier for the page (used by Flutter's diffing algorithm)
  - `path`: The URL path for the page
  - `title`: The title for the page (useful for AppBar or browser tab title)
  - `arguments`: Optional data for the page
  - `isHome` and `isTabPage`: Convenience properties for checking the page type

- **Route parsing with query parameters**: The parser can handle query parameters in the URL (e.g., `/search?q=flutter`). These are accessible via `uri.queryParameters`.

- **Tab switching logic**: The `switchTab()` method replaces the entire stack with a single tab page. This is typical for tab-based navigation where each tab has its own navigation stack.

- **Navigation methods**:
  - `pushPage()`: Adds a new page to the stack
  - `popPage()`: Removes the top page, updating the selected tab if needed
  - `replaceAll()`: Replaces the entire stack with a single page
  - `switchTab()`: Switches to a tab, replacing the stack with the tab's page
  - `pushTabPage()`: Pushes a tab page, either replacing or pushing depending on the current state

- **`onPopPage` with `route.didPop()`**: In addition to returning `true`, we call `route.didPop(result)` to properly handle the route removal. This is important for animations and state management.

- **Accessing the delegate in screens**: Screens use `Router.of(context).routerDelegate` to access the delegate and trigger navigation. This allows screens to push, pop, and manipulate the navigation stack.

---

## **17.3 Deep Linking Configuration (Android)**

Deep linking allows external apps and websites to open specific screens in your app. On Android, this is configured using App Links.

### **Android Manifest Configuration**

```dart
// ============================================
// FLUTTER CODE
// ============================================

import 'package:flutter/material.dart';

void main() {
  runApp(const DeepLinkingApp());
}

class DeepLinkingApp extends StatelessWidget {
  const DeepLinkingApp({super.key});

  @override
  Widget build(BuildContext context) {
    return MaterialApp.router(
      title: 'Deep Linking Demo',
      routerDelegate: DeepLinkRouterDelegate(),
      routeInformationParser: DeepLinkRouteInformationParser(),
    );
  }
}

// ... (RouterDelegate and RouteInformationParser implementations from previous sections)
```

```xml
<!-- ============================================ -->
<!-- ANDROID MANIFEST FILE -->
<!-- Location: android/app/src/main/AndroidManifest.xml -->
<!-- ============================================ -->

<manifest xmlns:android="http://schemas.android.com/apk/res/android"
    package="com.example.deep_linking_demo">

    <!-- Permissions for deep linking -->
    <uses-permission android:name="android.permission.INTERNET" />

    <application
        android:label="Deep Linking Demo"
        android:name="${applicationName}"
        android:icon="@mipmap/ic_launcher">
        
        <!-- ============================================ -->
        <!-- ACTIVITY DECLARATION WITH DEEP LINKING -->
        <!-- ============================================ -->
        <activity
            android:name=".MainActivity"
            android:exported="true"
            android:launchMode="singleTask"
            android:theme="@style/LaunchTheme"
            android:configChanges="orientation|keyboardHidden|keyboard|screenSize|smallestScreenSize|locale|layoutDirection|fontScale|screenLayout|density|uiMode"
            android:hardwareAccelerated="true"
            android:windowSoftInputMode="adjustResize">
            
            <!-- ============================================ -->
            <!-- INTENT FILTER FOR HTTP/HTTPS SCHEMES -->
            <!-- This allows your app to be opened from web links -->
            <!-- ============================================ -->
            <intent-filter android:autoVerify="true">
                <!-- autoVerify="true" enables Android App Links verification -->
                <!-- Android will verify that you own the domain by checking -->
                <!-- for a digital asset links file at yourdomain.com/.well-known/assetlinks.json -->
                
                <action android:name="android.intent.action.VIEW" />
                <!-- VIEW action indicates this intent should display data to the user -->
                
                <category android:name="android.intent.category.DEFAULT" />
                <!-- DEFAULT category allows the intent to be started without explicit component -->
                
                <category android:name="android.intent.category.BROWSABLE" />
                <!-- BROWSABLE category allows the intent to be started from a browser -->
                
                <!-- Accept HTTP links -->
                <data
                    android:scheme="http"
                    android:host="example.com"
                    android:pathPrefix="/" />
                <!-- This matches links like: http://example.com/anything -->
                
                <!-- Accept HTTPS links -->
                <data
                    android:scheme="https"
                    android:host="example.com"
                    android:pathPrefix="/" />
                <!-- This matches links like: https://example.com/anything -->
                
                <!-- You can have multiple data tags for different patterns -->
                <!-- For example, specific paths: -->
                <!--
                <data
                    android:scheme="https"
                    android:host="example.com"
                    android:path="/product" />
                <data
                    android:scheme="https"
                    android:host="example.com"
                    android:path="/profile" />
                -->
            </intent-filter>
            
            <!-- ============================================ -->
            <!-- INTENT FILTER FOR CUSTOM SCHEMES -->
            <!-- This allows your app to be opened from custom URLs -->
            <!-- ============================================ -->
            <intent-filter>
                <action android:name="android.intent.action.VIEW" />
                
                <category android:name="android.intent.category.DEFAULT" />
                <category android:name="android.intent.category.BROWSABLE" />
                
                <!-- Custom scheme: myapp:// -->
                <data
                    android:scheme="myapp"
                    android:host="product" />
                <!-- This matches links like: myapp://product/123 -->
                
                <data
                    android:scheme="myapp"
                    android:host="profile" />
                <!-- This matches links like: myapp://profile/user/123 -->
                
                <data
                    android:scheme="myapp"
                    android:host="settings" />
                <!-- This matches links like: myapp://settings -->
            </intent-filter>
        </activity>
        
        <!-- Don't delete the meta-data below.
             This is used by the Flutter tool to generate GeneratedPluginRegistrant.java -->
        <meta-data
            android:name="flutterEmbedding"
            android:value="2" />
    </application>
</manifest>
```

**Explanation:**

- **`<intent-filter>`**: Defines the conditions under which your app can be opened. Each `<intent-filter>` specifies:
  - `android:autoVerify="true"`: Enables App Links verification. Android will verify that you own the domain by checking for a digital asset links file. If verification fails, the intent filter still works as a regular deep link (without the "Open with [Your App]" dialog).
  - `<action android:name="android.intent.action.VIEW" />`: Specifies that this activity should handle viewing content (the most common action for deep links).
  - `<category android:name="android.intent.category.DEFAULT" />`: Allows the intent to be started without explicitly specifying the component.
  - `<category android:name="android.intent.category.BROWSABLE" />`: Allows the intent to be started from a browser (essential for web deep links).

- **HTTP/HTTPS schemes**: These intent filters handle standard web URLs. When a user clicks a link like `https://example.com/product/123` in a browser or another app, Android will check if your app can handle it.
  - `android:scheme="http"` or `"https"`: The URL scheme (protocol).
  - `android:host="example.com"`: The domain name. Only links to this domain will open your app.
  - `android:pathPrefix="/"`: The path prefix. All paths starting with `/` will match. You can be more specific (e.g., `"/product"`) to only match certain paths.

- **Custom schemes**: These intent filters handle custom URL schemes (like `myapp://`). Custom schemes don't require domain verification, making them easier to set up, but they don't provide the security and user experience benefits of HTTP/HTTPS App Links.
  - `android:scheme="myapp"`: The custom scheme. Choose a unique scheme name to avoid conflicts with other apps.
  - `android:host="product"`: The host part of the URL. Combined with the scheme, this matches URLs like `myapp://product/123`.

- **`android:exported="true"`: Allows the activity to be launched by other apps. This is required for deep linking to work.

- **`android:launchMode="singleTask"`**: Ensures only one instance of the activity exists. If the user clicks a deep link while the app is already open, the existing instance is reused instead of creating a new one. This is important for maintaining navigation state.

- **Multiple data tags**: You can include multiple `<data>` tags in a single `<intent-filter>`. They work with AND logic: a link must match ALL the data tags to open the app. For OR logic (multiple patterns), use multiple `<intent-filter>` blocks.

- **Domain verification**: For HTTP/HTTPS App Links, you must host a verification file at `https://<domain>/.well-known/assetlinks.json`. This file proves you own the domain. Without it, Android will still open deep links, but it will show a disambiguation dialog asking the user which app to use (your app or the browser).

### **Digital Asset Links File**

Create a JSON file at `https://example.com/.well-known/assetlinks.json` to verify domain ownership.

```json
[
  {
    "relation": [
      "delegate_permission/common.handle_all_urls"
    ],
    "target": {
      "namespace": "android_app",
      "package_name": "com.example.deep_linking_demo",
      "sha256_cert_fingerprints": [
        "AB:CD:EF:12:34:56:78:90:AB:CD:EF:12:34:56:78:90:AB:CD:EF:12:34:56:78:90:AB:CD:EF:12:34"
      ]
    }
  }
]
```

**Explanation:**

- **`relation`**: Specifies the permission being granted. `"delegate_permission/common.handle_all_urls"` means your app is authorized to handle all URLs for the specified domain.

- **`target.namespace`**: The platform for the target. `"android_app"` indicates this is for an Android app.

- **`target.package_name`**: Your app's package name (from `AndroidManifest.xml`). Must match exactly.

- **`target.sha256_cert_fingerprints`**: The SHA-256 fingerprint of your app's signing certificate. This is used to verify that the app requesting the permission is actually yours. You can obtain this fingerprint using:
  ```bash
  keytool -list -v -keystore <path-to-keystore> -alias <alias-name>
  ```
  Look for the "SHA256" line in the output.

- **Multiple apps**: You can include multiple entries in the array to support multiple apps or multiple signing certificates (e.g., debug and release builds).

- **Testing**: You can test your asset links file using Google's Digital Asset Links tool: https://developers.google.com/digital-asset-links/tools/generator

- **File location**: The file must be accessible at `https://<your-domain>/.well-known/assetlinks.json`:
  - Must use HTTPS (HTTP is not supported)
  - Must be publicly accessible (no authentication required)
  - Must return `Content-Type: application/json` header
  - Should have CORS enabled to allow access from any origin

### **Handling Incoming Deep Links in Flutter**

```dart
import 'package:flutter/material.dart';
import 'package:flutter/services.dart'; // For MethodChannel

void main() {
  runApp(const DeepLinkingApp());
}

class DeepLinkingApp extends StatefulWidget {
  const DeepLinkingApp({super.key});

  @override
  State<DeepLinkingApp> createState() => _DeepLinkingAppState();
}

class _DeepLinkingAppState extends State<DeepLinkingApp> {
  // MethodChannel for communicating with native Android/iOS code
  static const platform = MethodChannel('com.example.deep_linking_demo/links');
  
  String? _initialLink;
  String? _latestLink;

  @override
  void initState() {
    super.initState();
    _initDeepLinks();
  }

  Future<void> _initDeepLinks() async {
    try {
      // Get the initial link (if the app was opened from a deep link)
      // This covers the case where the app was NOT running when the link was clicked
      
      final initialLink = await platform.invokeMethod<String>('getInitialLink');
      print('Initial link: $initialLink');
      
      setState(() {
        _initialLink = initialLink;
      });
      
      // Handle the initial link
      if (initialLink != null) {
        _handleDeepLink(initialLink);
      }
    } on PlatformException catch (e) {
      print('Error getting initial link: ${e.message}');
    }
    
    // Listen for new deep links while the app is running
    // This covers the case where the app WAS running when the link was clicked
    
    platform.setMethodCallHandler((call) async {
      if (call.method == 'onNewLink') {
        final String link = call.arguments as String;
        print('New link: $link');
        
        setState(() {
          _latestLink = link;
        });
        
        // Handle the new link
        _handleDeepLink(link);
      }
    });
  }
  
  void _handleDeepLink(String link) {
    print('Handling deep link: $link');
    
    // Parse the link and navigate to the appropriate screen
    // You should integrate this with your RouterDelegate
    
    // Example parsing logic:
    final uri = Uri.parse(link);
    
    if (uri.scheme == 'myapp') {
      // Custom scheme: myapp://product/123
      final path = uri.path;
      
      if (path.startsWith('/product/')) {
        final productId = path.split('/')[2];
        print('Navigating to product: $productId');
        // TODO: Navigate to product screen using your RouterDelegate
      } else if (path.startsWith('/profile/')) {
        final userId = path.split('/')[2];
        print('Navigating to profile: $userId');
        // TODO: Navigate to profile screen using your RouterDelegate
      }
    } else if (uri.scheme == 'https' || uri.scheme == 'http') {
      // HTTP/HTTPS scheme: https://example.com/product/123
      final path = uri.path;
      
      if (path.startsWith('/product/')) {
        final productId = path.split('/')[2];
        print('Navigating to product: $productId');
        // TODO: Navigate to product screen using your RouterDelegate
      } else if (path.startsWith('/profile/')) {
        final userId = path.split('/')[2];
        print('Navigating to profile: $userId');
        // TODO: Navigate to profile screen using your RouterDelegate
      }
    }
  }

  @override
  Widget build(BuildContext context) {
    return MaterialApp.router(
      title: 'Deep Linking Demo',
      routerDelegate: DeepLinkRouterDelegate(
        initialLink: _initialLink,
      ),
      routeInformationParser: DeepLinkRouteInformationParser(),
    );
  }
}

// ============================================
// ANDROID NATIVE CODE (MainActivity.kt)
// Location: android/app/src/main/kotlin/com/example/deep_linking_demo/MainActivity.kt
// ============================================

package com.example.deep_linking_demo

import android.content.Intent
import android.os.Bundle
import io.flutter.embedding.android.FlutterActivity
import io.flutter.embedding.engine.FlutterEngine
import io.flutter.plugin.common.MethodChannel

class MainActivity: FlutterActivity() {
    // MethodChannel for communicating with Flutter
    private val CHANNEL = "com.example.deep_linking_demo/links"
    
    // Store the initial link (received in onCreate or onNewIntent)
    private var initialLink: String? = null
    
    // Flag to indicate if we've already sent the initial link to Flutter
    private var initialLinkSent = false

    override fun onCreate(savedInstanceState: Bundle?) {
        super.onCreate(savedInstanceState)
        
        // Get the intent that started the activity
        // This contains the deep link data if the app was opened from a link
        val intent = intent
        val data = intent?.data
        
        // Extract the deep link from the intent
        if (data != null) {
            initialLink = data.toString()
            println("Initial deep link: $initialLink")
        }
    }

    override fun configureFlutterEngine(flutterEngine: FlutterEngine) {
        super.configureFlutterEngine(flutterEngine)
        
        // Set up the MethodChannel
        MethodChannel(flutterEngine.dartExecutor.binaryMessenger, CHANNEL)
            .setMethodCallHandler { call, result ->
                if (call.method == "getInitialLink") {
                    // Return the initial link to Flutter
                    result.success(initialLink)
                    initialLinkSent = true
                } else {
                    result.notImplemented()
                }
            }
    }

    override fun onNewIntent(intent: Intent) {
        super.onNewIntent(intent)
        
        // onNewIntent is called when a new intent is received
        // while the activity is already running
        // This happens when the user clicks a deep link while the app is open
        
        val data = intent?.data
        if (data != null) {
            val newLink = data.toString()
            println("New deep link: $newLink")
            
            // Send the new link to Flutter
            val flutterEngine = flutterEngine ?: return
            MethodChannel(flutterEngine.dartExecutor.binaryMessenger, CHANNEL)
                .invokeMethod("onNewLink", newLink)
        }
    }
}
```

**Explanation:**

- **Flutter side (`DeepLinkingApp`)**:
  - **`MethodChannel`**: A channel for communication between Flutter and native Android code. The channel name must match on both sides (`'com.example.deep_linking_demo/links'`).
  - **`getInitialLink`**: Calls the native method to get the initial deep link (received when the app was opened from a link). This covers the case where the app was NOT running when the user clicked the link.
  - **`onNewLink` handler**: Listens for new deep links received while the app is running. This covers the case where the app WAS running when the user clicked the link.
  - **`_handleDeepLink`**: Parses the deep link and determines which screen to navigate to. This should be integrated with your `RouterDelegate` to actually perform the navigation.
  - **Link parsing**: The code checks both custom schemes (`myapp://`) and HTTP/HTTPS schemes (`https://`). It extracts parameters from the path (e.g., product ID from `/product/123`).

- **Android native side (`MainActivity.kt`)**:
  - **`onCreate`**: Called when the activity is first created. This is where we extract the initial deep link from the intent. The intent contains the data (URL) that started the activity.
  - **`intent?.data`**: The `data` property of the intent contains the deep link URL. It's a `Uri` object that can be converted to a string.
  - **`configureFlutterEngine`**: Sets up the `MethodChannel` to handle calls from Flutter. We implement the `getInitialLink` method to return the stored initial link.
  - **`onNewIntent`**: Called when a new intent is received while the activity is already running. This happens when the user clicks a deep link while the app is open. We extract the new link and send it to Flutter using `invokeMethod`.
  - **`flutterEngine`**: The Flutter engine that's running in the activity. We use it to access the `MethodChannel` for communication with Flutter.
  - **`invokeMethod`**: Sends a message from native code to Flutter. We call `onNewLink` with the new link as the argument.

- **Testing deep links**: You can test deep links using:
  - **ADB command**: `adb shell am start -W -a android.intent.action.VIEW -d "https://example.com/product/123" com.example.deep_linking_demo`
  - **Browser**: Type the URL in a mobile browser and click it
  - **Another app**: Create a link in another app (like a note-taking app) and click it
  - **SMS/Email**: Send yourself a deep link via SMS or email and click it

- **Common issues**:
  - **App not opening**: Check that `android:exported="true"` is set in the manifest
  - **Multiple apps handling the link**: If multiple apps handle the same URL scheme, Android will show a disambiguation dialog. Use unique schemes or set up App Links verification.
  - **Link not received in Flutter**: Ensure that the `MethodChannel` names match on both sides and that the native code properly invokes the methods.

---

## **17.4 Deep Linking Configuration (iOS)**

On iOS, deep linking is configured using Universal Links, which provide a secure way to open your app from web links.

### **iOS Configuration Files**

```xml
<!-- ============================================ -->
<!-- INFO.PLIST FILE -->
<!-- Location: ios/Runner/Info.plist -->
<!-- ============================================ -->

<?xml version="1.0" encoding="UTF-8"?>
<!DOCTYPE plist PUBLIC "-//Apple//DTD PLIST 1.0//EN" "http://www.apple.com/DTDs/PropertyList-1.0.dtd">
<plist version="1.0">
<dict>
	<!-- ... existing configuration ... -->
	
	<!-- ============================================ -->
	<!-- URL SCHEMES (CUSTOM DEEP LINKS) -->
	<!-- ============================================ -->
	<key>CFBundleURLTypes</key>
	<array>
		<!-- First URL scheme: myapp -->
		<dict>
			<key>CFBundleTypeRole</key>
			<string>Editor</string>
			
			<key>CFBundleURLName</key>
			<string>com.example.deep_linking_demo</string>
			<!-- Unique identifier for this URL type -->
			
			<key>CFBundleURLSchemes</key>
			<array>
				<string>myapp</string>
				<!-- The custom scheme (without ://) -->
				<!-- This matches URLs like: myapp://product/123 -->
			</array>
		</dict>
		
		<!-- You can add multiple URL schemes -->
		<!-- Example: another scheme -->
		<!--
		<dict>
			<key>CFBundleTypeRole</key>
			<string>Editor</string>
			
			<key>CFBundleURLName</key>
			<string>com.example.deep_linking_demo.another</string>
			
			<key>CFBundleURLSchemes</key>
			<array>
				<string>anotherapp</string>
			</array>
		</dict>
		-->
	</array>
	
	<!-- ============================================ -->
	<!-- UNIVERSAL LINKS (HTTP/HTTPS DEEP LINKS) -->
	<!-- ============================================ -->
	<key>com.apple.developer.associated-domains</key>
	<array>
		<!-- Each entry specifies a domain and path pattern -->
		<!-- Format: applinks:domain[:path] -->
		
		<!-- Universal Links for example.com -->
		<string>applinks:example.com</string>
		<!-- This matches all paths on example.com -->
		<!-- E.g., https://example.com/product/123 -->
		
		<!-- You can specify path patterns for more control -->
		<!-- Examples: -->
		<!--
		<string>applinks:example.com/product/*</string>
		<string>applinks:example.com/profile/*</string>
		-->
		
		<!-- Multiple domains -->
		<!--
		<string>applinks:www.example.com</string>
		<string>applinks:api.example.com</string>
		-->
	</array>
</dict>
</plist>
```

**Explanation:**

- **`CFBundleURLTypes`**: Array of URL types that your app can handle. Each entry defines:
  - **`CFBundleTypeRole`**: The role of your app for this URL type. `"Editor"` means your app can both view and edit the content. Other values include `"Viewer"` (view only), `"None"` (no specific role).
  - **`CFBundleURLName`**: A unique identifier for this URL type. Typically uses reverse DNS notation (e.g., `com.example.app`).
  - **`CFBundleURLSchemes`**: Array of URL schemes (without `://`). For example, `myapp` matches URLs like `myapp://product/123`. Custom schemes are simpler to set up than Universal Links but don't provide the same security and user experience.

- **Custom schemes vs. Universal Links**:
  - **Custom schemes**: Easier to set up (no domain verification required), but can have conflicts with other apps, and users may see an "Open in..." dialog. Best for app-specific URLs that won't be shared on the web.
  - **Universal Links**: Require domain verification but provide a seamless user experience (no dialog, links open directly in your app). Best for web URLs that should open in your app.

- **`com.apple.developer.associated-domains`**: Array of domains for Universal Links. Each entry:
  - Starts with `applinks:` to indicate it's for Universal Links.
  - Specifies the domain (e.g., `example.com`).
  - Can optionally include a path pattern (e.g., `example.com/product/*`).
  - The path pattern uses wildcard matching:
    - `*`: Matches any characters (including none)
    - `?`: Matches any single character
    - Examples: `/product/*` matches `/product/123` but not `/product`; `/product/?` matches `/product/1` but not `/product/12`.

- **Path patterns**: You can specify path patterns to restrict which paths open your app:
  - `applinks:example.com`: All paths on `example.com` open your app
  - `applinks:example.com/product/*`: Only paths starting with `/product/` open your app
  - `applinks:example.com/product/123`: Only the exact path `/product/123` opens your app

- **Multiple domains**: You can specify multiple domains if your app handles links from multiple websites. Each domain requires its own Apple App Site Association file.

### **Apple App Site Association File**

Create a JSON file at `https://example.com/.well-known/apple-app-site-association` to verify domain ownership.

```json
{
  "applinks": {
    "apps": [],
    "details": [
      {
        "appIDs": [
          "TEAMID.com.example.deep-linking-demo"
        ],
        "components": [
          {
            "/": "/product/*",
            "comment": "Matches all product pages"
          },
          {
            "/": "/profile/*",
            "comment": "Matches all profile pages"
          },
          {
            "/": "/settings",
            "comment": "Matches the settings page"
          }
        ]
      }
    ]
  },
  "webcredentials": {
    "apps": [
      "TEAMID.com.example.deep-linking-demo"
    ]
  }
}
```

**Explanation:**

- **File location and requirements**:
  - Must be hosted at `https://<your-domain>/.well-known/apple-app-site-association`
  - Must use HTTPS (HTTP is not supported)
  - Must not have a file extension (the `.json` extension is implied but should not be in the URL)
  - Must return `Content-Type: application/json` header
  - Must be publicly accessible (no authentication required)
  - Should have a small file size (Apple recommends keeping it under 10KB)
  - Must be valid JSON (no trailing commas, quotes around all keys and string values)

- **`applinks` section**: Defines Universal Links configuration:
  - **`apps`**: Array of app IDs that should NOT be checked for Universal Links. Typically empty (`[]`), as you want to check all apps. This is a legacy field that should be left empty.
  - **`details`**: Array of configuration objects for each app:
    - **`appIDs`**: Array of app IDs in the format `TEAMID.bundle.identifier`. You can find your Team ID in the Apple Developer portal, and your bundle identifier in Xcode (under "Signing & Capabilities").
    - **`components`**: Array of path component rules:
      - **`/`**: The path pattern to match. Supports wildcards (`*` for any characters, `?` for any single character).
      - **`comment`**: Optional description for documentation purposes.
      - **`exclude`**: Optional boolean. If `true`, this path pattern is excluded (not matched). Useful for excluding certain paths while including others.
      - **`query`**: Optional object for matching query parameters. Example:
        ```json
        {
          "/": "/product",
          "query": {
            "id": "?*"
          }
        }
        ```
        This matches `/product?id=<anything>`.

- **`webcredentials` section**: Defines which app handles password autofill and web credentials:
  - **`apps`**: Array of app IDs (same format as `appIDs` in `applinks`). This enables your app to handle web credentials for the domain, allowing users to save passwords in your app's Keychain.

- **Finding your Team ID**:
  - Go to the Apple Developer portal (https://developer.apple.com/account)
  - Look at the top-right corner under your name
  - Your Team ID is a 10-character alphanumeric string (e.g., `ABC1234567`)

- **Finding your bundle identifier**:
  - Open your project in Xcode
  - Select the target (Runner)
  - Go to the "General" tab
  - Look for "Bundle Identifier" under "Identity"

- **Multiple apps**: You can include multiple apps in the `details` array. This is useful if you have multiple apps (e.g., main app and lite version) that should handle the same domains.

- **Testing**: You can test your Apple App Site Association file using:
  - Apple's validator tool: https://developer.apple.com/associated-domains/entitlement-validator
  - Browser: Visit `https://example.com/.well-known/apple-app-site-association` and verify the JSON is valid
  - Command line: `curl -v https://example.com/.well-known/apple-app-site-association`

### **Handling Incoming Deep Links in iOS**

```dart
import 'package:flutter/material.dart';
import 'package:flutter/services.dart';

void main() {
  runApp(const DeepLinkingIOSApp());
}

class DeepLinkingIOSApp extends StatefulWidget {
  const DeepLinkingIOSApp({super.key});

  @override
  State<DeepLinkingIOSApp> createState() => _DeepLinkingIOSAppState();
}

class _DeepLinkingIOSAppState extends State<DeepLinkingIOSApp> {
  static const platform = MethodChannel('com.example.deep_linking_demo/links');
  
  String? _initialLink;
  String? _latestLink;

  @override
  void initState() {
    super.initState();
    _initDeepLinks();
  }

  Future<void> _initDeepLinks() async {
    try {
      // Get the initial link (if the app was opened from a deep link)
      final initialLink = await platform.invokeMethod<String>('getInitialLink');
      print('Initial link (iOS): $initialLink');
      
      setState(() {
        _initialLink = initialLink;
      });
      
      // Handle the initial link
      if (initialLink != null) {
        _handleDeepLink(initialLink);
      }
    } on PlatformException catch (e) {
      print('Error getting initial link: ${e.message}');
    }
    
    // Listen for new deep links while the app is running
    platform.setMethodCallHandler((call) async {
      if (call.method == 'onNewLink') {
        final String link = call.arguments as String;
        print('New link (iOS): $link');
        
        setState(() {
          _latestLink = link;
        });
        
        // Handle the new link
        _handleDeepLink(link);
      }
    });
  }
  
  void _handleDeepLink(String link) {
    print('Handling deep link (iOS): $link');
    
    final uri = Uri.parse(link);
    
    if (uri.scheme == 'myapp') {
      // Custom scheme: myapp://product/123
      final path = uri.path;
      
      if (path.startsWith('/product/')) {
        final productId = path.split('/')[2];
        print('Navigating to product: $productId');
        // TODO: Navigate using your RouterDelegate
      } else if (path.startsWith('/profile/')) {
        final userId = path.split('/')[2];
        print('Navigating to profile: $userId');
        // TODO: Navigate using your RouterDelegate
      }
    } else if (uri.scheme == 'https' || uri.scheme == 'http') {
      // Universal Link: https://example.com/product/123
      final path = uri.path;
      
      if (path.startsWith('/product/')) {
        final productId = path.split('/')[2];
        print('Navigating to product: $productId');
        // TODO: Navigate using your RouterDelegate
      } else if (path.startsWith('/profile/')) {
        final userId = path.split('/')[2];
        print('Navigating to profile: $userId');
        // TODO: Navigate using your RouterDelegate
      }
    }
  }

  @override
  Widget build(BuildContext context) {
    return MaterialApp.router(
      title: 'Deep Linking iOS Demo',
      routerDelegate: DeepLinkRouterDelegate(
        initialLink: _initialLink,
      ),
      routeInformationParser: DeepLinkRouteInformationParser(),
    );
  }
}
```

```swift
// ============================================
// IOS NATIVE CODE (AppDelegate.swift)
// Location: ios/Runner/AppDelegate.swift
// ============================================

import UIKit
import Flutter
import UniformTypeIdentifiers

@UIApplicationMain
@objc class AppDelegate: FlutterAppDelegate {
    // MethodChannel for communicating with Flutter
    private let channel = "com.example.deep_linking_demo/links"
    
    // Flag to indicate if we've already sent the initial link to Flutter
    private var initialLinkSent = false
    
    // Store the Flutter method channel for later use
    private var methodChannel: FlutterMethodChannel?

    override func application(
        _ application: UIApplication,
        didFinishLaunchingWithOptions launchOptions: [UIApplication.LaunchOptionsKey: Any]?
    ) -> Bool {
        // Get the Flutter engine controller
        guard let controller = window?.rootViewController as? FlutterViewController else {
            fatalError("rootViewController is not FlutterViewController")
        }
        
        // Set up the MethodChannel
        methodChannel = FlutterMethodChannel(
            name: channel,
            binaryMessenger: controller.binaryMessenger
        )
        
        // Set up the method call handler
        methodChannel?.setMethodCallHandler { [weak self] (call, result) in
            guard let self = self else { return }
            
            switch call.method {
            case "getInitialLink":
                // Return the initial link to Flutter
                result(self.getInitialLink())
                self.initialLinkSent = true
            default:
                result(FlutterMethodNotImplemented)
            }
        }
        
        return super.application(application, didFinishLaunchingWithOptions: launchOptions)
    }
    
    // Get the initial link from launch options
    private func getInitialLink() -> String? {
        // Check if the app was launched from a deep link
        // This covers the case where the app was NOT running when the link was clicked
        
        // Check for Universal Links (https://)
        if let userActivity = userActivity,
           let webpageURL = userActivity.webpageURL {
            print("Initial Universal Link: \(webpageURL.absoluteString)")
            return webpageURL.absoluteString
        }
        
        // Check for custom schemes (myapp://)
        // For custom schemes, the URL is in the launch options under .url
        if let url = launchOptions?[.url] as? URL {
            print("Initial Custom Scheme Link: \(url.absoluteString)")
            return url.absoluteString
        }
        
        return nil
    }
    
    // Handle Universal Links (https://) when the app is already running
    override func application(
        _ application: UIApplication,
        continue userActivity: NSUserActivity,
        restorationHandler: @escaping ([UIUserActivityRestoring]?) -> Void
    ) -> Bool {
        // This method is called when the app is already running
        // and a Universal Link is clicked
        
        guard let url = userActivity.webpageURL else {
            return false
        }
        
        print("New Universal Link: \(url.absoluteString)")
        
        // Send the link to Flutter
        methodChannel?.invokeMethod("onNewLink", arguments: url.absoluteString)
        
        return true
    }
    
    // Handle custom schemes (myapp://) when the app is already running
    // This is handled by openURL below
}
```

**Explanation:**

- **Flutter side (`DeepLinkingIOSApp`)**:
  - Similar to the Android implementation, uses a `MethodChannel` to communicate with native iOS code.
  - `getInitialLink`: Retrieves the initial deep link (received when the app was opened from a link).
  - `onNewLink` handler: Listens for new deep links received while the app is running.
  - `_handleDeepLink`: Parses the link and navigates to the appropriate screen.

- **iOS native side (`AppDelegate.swift`)**:
  - **`@UIApplicationMain`**: Marks this class as the app delegate, which receives app-level events.
  - **`application(_:didFinishLaunchingWithOptions:)`**: Called when the app finishes launching. We set up the `FlutterMethodChannel` here and register the method call handler.
  - **`FlutterViewController`**: The root view controller for Flutter apps. We cast the root view controller to this type to access the Flutter engine.
  - **`FlutterMethodChannel`**: Channel for communication between Flutter and iOS native code. The channel name must match the Flutter side.
  - **`setMethodCallHandler`**: Registers a handler for method calls from Flutter. We handle the `getInitialLink` method.
  - **`getInitialLink()`**: Extracts the initial deep link from `userActivity` (for Universal Links) or `launchOptions[.url]` (for custom schemes). `userActivity.webpageURL` contains the URL for Universal Links.
  - **`application(_:continue:restorationHandler:)`**: Called when a Universal Link is received while the app is already running. We extract the URL and send it to Flutter using `invokeMethod`.

- **Universal Links vs. Custom Schemes on iOS**:
  - **Universal Links**: Handled by `application(_:continue:restorationHandler:)`. The URL is in `userActivity.webpageURL`. Universal Links use the standard `https://` scheme.
  - **Custom schemes**: Handled by `openURL` (not shown here) or accessed via `launchOptions[.url]` when the app is first launched. Custom schemes use custom schemes like `myapp://`.

- **Testing deep links on iOS**:
  - **Simulator**: Use the Simulator's "Deep Link" feature:
    - Open Safari in the simulator
    - Type the deep link in the address bar
    - Press Enter to navigate
  - **Xcode**: In Xcode, go to "Product" → "Scheme" → "Edit Scheme" → "Run" → "Options" → "Launch". Add your deep link in the "App URL" field.
  - **Device**: Use Notes, Mail, or Safari to create and click deep links.

- **Common issues**:
  - **Universal Links not working**: Verify that the Apple App Site Association file is valid and accessible. Check that the app ID in the file matches your Team ID and bundle identifier.
  - **Custom schemes not working**: Ensure that `CFBundleURLTypes` is correctly configured in `Info.plist`.
  - **Link not received in Flutter**: Check that the `MethodChannel` names match and that the native code properly invokes the methods.

---

## **Chapter Summary**

In this chapter, we covered deep linking and Navigation 2.0 in Flutter:

### **Key Takeaways:**

1. **Navigator 2.0 Overview**:
   - Declarative navigation API that provides full control over the navigation stack
   - Enables URL syncing for web support
   - Simplifies deep linking implementation
   - Navigation state is data-driven and predictable

2. **Core Components**:
   - **`RouterDelegate`**: Builds and manages the Navigator based on navigation state
   - **`RouteInformationParser`**: Converts URLs/RouteInformation into navigation state
   - **`backButtonDispatcher`**: Handles back button presses (Android)
   - **Navigation state**: Custom data structure representing the navigation stack

3. **Declarative Routing**:
   - Define navigation state as data (e.g., list of pages)
   - Use `MaterialApp.router()` instead of `MaterialApp`
   - Implement `build()`, `currentConfiguration`, `setInitialRoutePath()`, and `setNewRoutePath()`
   - Mix with `ChangeNotifier` and `PopNavigatorRouterDelegateMixin`

4. **URL Support**:
   - Parse URLs from `RouteInformation.uri`
   - Convert navigation state back to URLs for browser updates
   - Handle query parameters and path parameters
   - Support both hash-based and path-based routing

5. **Android Deep Linking**:
   - Configure `AndroidManifest.xml` with `<intent-filter>` entries
   - Support HTTP/HTTPS schemes (App Links) with domain verification
   - Support custom schemes (e.g., `myapp://`)
   - Use `MainActivity.onCreate()` and `onNewIntent()` to handle deep links
   - Host digital asset links file at `/.well-known/assetlinks.json`

6. **iOS Deep Linking**:
   - Configure `Info.plist` with `CFBundleURLTypes` and `com.apple.developer.associated-domains`
   - Support custom schemes (`CFBundleURLSchemes`)
   - Support Universal Links (`applinks:`)
   - Use `AppDelegate` methods to handle deep links
   - Host Apple App Site Association file at `/.well-known/apple-app-site-association`

7. **Platform Integration**:
   - Use `MethodChannel` for Flutter-native communication
   - Handle initial links (app not running when link clicked)
   - Handle new links (app running when link clicked)
   - Parse links and integrate with `RouterDelegate`

8. **Universal Links vs. Custom Schemes**:
   - **Universal Links**: Require domain verification, provide seamless UX, use standard HTTP/HTTPS
   - **Custom Schemes**: Easier to set up, may show disambiguation dialog, use custom schemes

### **Next Steps:**

Now that you understand deep linking and Navigation 2.0, the next chapter will cover GoRouter and advanced navigation patterns, including:

- GoRouter setup and configuration
- ShellRoute and nested navigation
- Redirects and guards
- Query parameters and path parameters
- Error handling and 404 pages

---

**End of Chapter 17**

---

# **Next Chapter: Chapter 18 - GoRouter & Advanced Navigation**

Chapter 18 will explore GoRouter, a powerful declarative routing library built on Navigator 2.0. You'll learn how to set up GoRouter, handle nested navigation with ShellRoute, implement redirects and guards, and manage complex navigation patterns with less boilerplate code.